In [1]:
import pandas as pd

In [2]:
beneficiary = pd.read_csv('Beneficiarydatafile.csv')

provider = pd.read_csv('Provider_Dataset.csv')
#remove claim id from columns
provider = provider.loc[:, provider.columns != 'Claim_ID']
#remove ClmProcedureCode_6 as all columns are blank
provider = provider.loc[:, provider.columns != 'ClmProcedureCode_6']

train_dataset = pd.read_excel('Train_Dataset.xlsx')
test_dataset = pd.read_excel('Test_Dataset.xlsx')

provider.isna().sum()

/Users/anmol/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AdmissionDate                  517737
Attended_Physician               1508
Beneficiary_ID                      0
Claim_End_Date                      0
Claim_Start_Date                    0
Clm_Admit_Code                 412312
ClmDiagnosisCode_1              10453
ClmDiagnosisCode_10            553201
ClmDiagnosisCode_2             195606
ClmDiagnosisCode_3             315156
ClmDiagnosisCode_4             393675
ClmDiagnosisCode_5             446287
ClmDiagnosisCode_6             473819
ClmDiagnosisCode_7             492034
ClmDiagnosisCode_8             504767
ClmDiagnosisCode_9             516396
ClmProcedureCode_1             534901
ClmProcedureCode_2             552721
ClmProcedureCode_3             557242
ClmProcedureCode_4             558093
ClmProcedureCode_5             558202
Deductible_Amount_Paid            899
Diagnosis_Group_Code           517737
Discharge_date                 517737
Insurance_Amount_Reimbursed         0
OperatingPhysician             443764
OtherPhysici

In [3]:
provider['Claim_Start_Date'] = pd.to_datetime(provider['Claim_Start_Date'])
provider['Claim_End_Date'] = pd.to_datetime(provider['Claim_End_Date'])

provider['AdmissionDate'] =  pd.to_datetime(provider['AdmissionDate'])
provider['Discharge_date'] = pd.to_datetime(provider['Discharge_date'])

#Fill all missing Admission date and discharge date with claim start and end
provider['AdmissionDate'].fillna(provider['Claim_Start_Date'], inplace=True)
provider['Discharge_date'].fillna(provider['Claim_End_Date'], inplace=True)

provider['Days stayed'] = (provider['Discharge_date'] - provider['AdmissionDate']).dt.days
provider['Days in claim'] = (provider['Claim_End_Date'] - provider['Claim_Start_Date']).dt.days
provider['Days Admission to Claim'] = (provider['Claim_Start_Date'] - provider['AdmissionDate']).dt.days

#drop date columns once done
provider.drop(columns=['Claim_Start_Date', 'Claim_End_Date', 'Discharge_date'], inplace=True)

In [4]:
#Fill missing operating physician with other physcian
provider['OperatingPhysician'].fillna(provider['OtherPhysician'], inplace=True)
provider['Attended_Physician'].fillna(provider['OperatingPhysician'], inplace=True)

#Fill backwards, as attending, operating, other physician could be same
provider['OperatingPhysician'].fillna(provider['Attended_Physician'], inplace=True)
provider['OperatingPhysician'].fillna(provider['OtherPhysician'], inplace=True)

provider['OtherPhysician'].fillna(provider['Attended_Physician'], inplace=True)
provider['OtherPhysician'].fillna(provider['OperatingPhysician'], inplace=True)

#Create a category "NA" for all missing physcian details
provider['Attended_Physician'].fillna('NA', inplace=True)
provider['OperatingPhysician'].fillna('NA', inplace=True)
provider['OtherPhysician'].fillna('NA', inplace=True)

#fill missing deductible amount paid
#Highly correlated to patient type, almost a correlation of 1
#all missing Deductible_Amount_Paid has patient type 0, ie value 1068
provider['Deductible_Amount_Paid'].fillna(1068, inplace=True)

#count no of diagnosis and no of procedures
provider['Diagnosis Count'] = provider[['ClmDiagnosisCode_1','ClmDiagnosisCode_2','ClmDiagnosisCode_3','ClmDiagnosisCode_4',
                         'ClmDiagnosisCode_5','ClmDiagnosisCode_6','ClmDiagnosisCode_7','ClmDiagnosisCode_8',
                         'ClmDiagnosisCode_9','ClmDiagnosisCode_10']].notnull().sum(axis=0)
provider['Procedure Count'] = provider[['ClmProcedureCode_1','ClmProcedureCode_2','ClmProcedureCode_3',
                         'ClmProcedureCode_4','ClmProcedureCode_5']].notnull().sum(axis=0)

provider.dtypes

AdmissionDate                  datetime64[ns]
Attended_Physician                     object
Beneficiary_ID                         object
Clm_Admit_Code                         object
ClmDiagnosisCode_1                     object
ClmDiagnosisCode_10                    object
ClmDiagnosisCode_2                     object
ClmDiagnosisCode_3                     object
ClmDiagnosisCode_4                     object
ClmDiagnosisCode_5                     object
ClmDiagnosisCode_6                     object
ClmDiagnosisCode_7                     object
ClmDiagnosisCode_8                     object
ClmDiagnosisCode_9                     object
ClmProcedureCode_1                    float64
ClmProcedureCode_2                    float64
ClmProcedureCode_3                    float64
ClmProcedureCode_4                    float64
ClmProcedureCode_5                    float64
Deductible_Amount_Paid                float64
Diagnosis_Group_Code                   object
Insurance_Amount_Reimbursed       

In [5]:
#Check count of null values in each of columns
provider.isna().sum()

provider = provider[['Beneficiary_ID',
                     'Deductible_Amount_Paid', 'Insurance_Amount_Reimbursed',
                     'Attended_Physician', 'OperatingPhysician', 'OtherPhysician',
                     'PatientType',
                     'Provider_Hospital',
                     'Days stayed', 'Days in claim', 'Days Admission to Claim',
                     'Diagnosis Count','Procedure Count',
                     'AdmissionDate']]

## Work on Beneficiary data

In [6]:
beneficiary.sample(5)
beneficiary.isna().sum()
beneficiary.dtypes

Beneficiary_ID                     object
DOB                                object
DOD                                object
Gender                              int64
Race                                int64
RenalDiseaseIndicator              object
State                               int64
County                              int64
NoOfMonths_PartACov                 int64
NoOfMonths_PartBCov                 int64
ChronicCond_Alzheimer               int64
ChronicCond_Heartfailure            int64
ChronicCond_KidneyDisease           int64
ChronicCond_Cancer                  int64
ChronicCond_ObstrPulmonary          int64
ChronicCond_Depression              int64
ChronicCond_Diabetes                int64
ChronicCond_IschemicHeart           int64
ChronicCond_Osteoporasis            int64
ChronicCond_rheumatoidarthritis     int64
ChronicCond_stroke                  int64
IPAnnualReimbursementAmt            int64
IPAnnualDeductibleAmt               int64
OPAnnualReimbursementAmt          

In [7]:
beneficiary['DOD'] = beneficiary['DOD'].fillna(0)
beneficiary['Died'] = beneficiary['DOD'].astype(bool).astype(int)
beneficiary.drop(columns=['DOD'], inplace=True)

#convert Y to 1 and all 2s to 0
beneficiary.loc[beneficiary['RenalDiseaseIndicator'] == 'Y', 'RenalDiseaseIndicator'] = 1
beneficiary.loc[beneficiary['ChronicCond_Alzheimer'] == 2, 'ChronicCond_Alzheimer'] = 0
beneficiary.loc[beneficiary['ChronicCond_Heartfailure'] == 2, 'ChronicCond_Heartfailure'] = 0
beneficiary.loc[beneficiary['ChronicCond_KidneyDisease'] == 2, 'ChronicCond_KidneyDisease'] = 0
beneficiary.loc[beneficiary['ChronicCond_Cancer'] == 2, 'ChronicCond_Cancer'] = 0
beneficiary.loc[beneficiary['ChronicCond_ObstrPulmonary'] == 2, 'ChronicCond_ObstrPulmonary'] = 0
beneficiary.loc[beneficiary['ChronicCond_Depression'] == 2, 'ChronicCond_Depression'] = 0
beneficiary.loc[beneficiary['ChronicCond_Diabetes'] == 2, 'ChronicCond_Diabetes'] = 0
beneficiary.loc[beneficiary['ChronicCond_IschemicHeart'] == 2, 'ChronicCond_IschemicHeart'] = 0
beneficiary.loc[beneficiary['ChronicCond_Osteoporasis'] == 2, 'ChronicCond_Osteoporasis'] = 0
beneficiary.loc[beneficiary['ChronicCond_rheumatoidarthritis'] == 2, 'ChronicCond_rheumatoidarthritis'] = 0
beneficiary.loc[beneficiary['ChronicCond_stroke'] == 2, 'ChronicCond_stroke'] = 0

In [8]:
beneficiary.sample(5)

,Beneficiary_ID,DOB,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Died
131688,BENE151832,01-02-1924,2,1,0,52,420,12,12,0,...,0,1,0,0,0,0,0,50,90,0
21444,BENE33965,01-02-1920,2,1,0,5,460,12,12,1,...,1,0,0,0,0,13000,1068,910,40,0
104026,BENE122289,01-06-1933,1,1,1,39,560,12,12,1,...,1,1,1,1,1,46720,2136,110,60,0
42393,BENE56390,01-08-1938,2,2,1,14,141,12,12,1,...,1,1,0,1,1,16260,2136,2360,570,0
34765,BENE48267,01-12-1941,2,1,0,14,141,12,12,0,...,0,1,0,1,0,0,0,1320,120,0


In [9]:
print(provider.shape)
print(beneficiary.shape)

#for simplicity inner join
data = pd.concat([provider,beneficiary], axis=1, join='inner')
print(data.shape)

(558211, 14)
(138556, 25)
(138556, 39)


In [10]:
data['Age']  = round(((pd.to_datetime(data['AdmissionDate']) - pd.to_datetime(data['DOB'])).dt.days)/365.25, 0)
data.drop(columns=['Beneficiary_ID','AdmissionDate','DOB'], inplace=True)

In [11]:
data.dtypes

Deductible_Amount_Paid             float64
Insurance_Amount_Reimbursed          int64
Attended_Physician                  object
OperatingPhysician                  object
OtherPhysician                      object
PatientType                          int64
Provider_Hospital                   object
Days stayed                          int64
Days in claim                        int64
Days Admission to Claim              int64
Diagnosis Count                    float64
Procedure Count                    float64
Gender                               int64
Race                                 int64
RenalDiseaseIndicator               object
State                                int64
County                               int64
NoOfMonths_PartACov                  int64
NoOfMonths_PartBCov                  int64
ChronicCond_Alzheimer                int64
ChronicCond_Heartfailure             int64
ChronicCond_KidneyDisease            int64
ChronicCond_Cancer                   int64
ChronicCond

In [12]:
df = pd.DataFrame(index = data.Provider_Hospital.unique())
df['Total Deductible Amount'] = data.groupby(['Provider_Hospital'])['Deductible_Amount_Paid'].agg('sum')
df['Avg Deductible Amount'] = data.groupby(['Provider_Hospital'])['Deductible_Amount_Paid'].agg('mean')
df['Total Insurance Amount'] = data.groupby(['Provider_Hospital'])['Insurance_Amount_Reimbursed'].agg('sum')
df['Avg Insurance Amount'] = data.groupby(['Provider_Hospital'])['Insurance_Amount_Reimbursed'].agg('mean')

df['Avg Days Stayed'] = data.groupby(['Provider_Hospital'])['Days stayed'].agg('mean')
df['Avg Days in claim'] = data.groupby(['Provider_Hospital'])['Days in claim'].agg('mean')

#This is coming as negative, fix required
df['Avg Days Admission to Claim'] = data.groupby(['Provider_Hospital'])['Days Admission to Claim'].agg('mean')
df['Avg Age'] = data.groupby(['Provider_Hospital'])['Age'].agg('mean')

#df['Avg Diagnosis Count'] = data.groupby(['Provider_Hospital'])['Diagnosis Count'].agg('mean')
#df['Procedure Count'] = data.groupby(['Provider_Hospital'])['Procedure Count'].agg('mean')

df['Provider'] = df.index
df.sample(5)

,Total Deductible Amount,Avg Deductible Amount,Total Insurance Amount,Avg Insurance Amount,Avg Days Stayed,Avg Days in claim,Avg Days Admission to Claim,Avg Age,Provider
PRV54296,0.0,0.000000,2260,1130.000000,0.500000,0.500000,0.0,61.500000,PRV54296
PRV54937,3274.0,363.777778,25250,2805.555556,-26.111111,-26.111111,0.0,73.555556,PRV54937
PRV56853,0.0,0.000000,100,50.000000,0.000000,0.000000,0.0,48.000000,PRV56853
PRV53013,0.0,0.000000,0,0.000000,0.000000,0.000000,0.0,66.000000,PRV53013
PRV54410,0.0,0.000000,460,65.714286,0.000000,0.000000,0.0,73.285714,PRV54410


In [13]:
data = pd.merge(df, train_dataset, on='Provider')
data['PotentialFraud'] = data.PotentialFraud.map(dict(Yes=1, No=0))
data.isna().sum()

Total Deductible Amount        0
Avg Deductible Amount          0
Total Insurance Amount         0
Avg Insurance Amount           0
Avg Days Stayed                0
Avg Days in claim              0
Avg Days Admission to Claim    0
Avg Age                        0
Provider                       0
PotentialFraud                 0
dtype: int64

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns

corr = data.corr()

# plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

In [15]:
data.sample(5)

,Total Deductible Amount,Avg Deductible Amount,Total Insurance Amount,Avg Insurance Amount,Avg Days Stayed,Avg Days in claim,Avg Days Admission to Claim,Avg Age,Provider,PotentialFraud
1211,3204.0,640.8,11100,2220.000000,37.000000,37.000000,0.0,77.200000,PRV54756,0
3018,0.0,0.0,690,86.250000,0.000000,0.000000,0.0,74.750000,PRV52384,0
960,23496.0,1068.0,213000,9681.818182,46.500000,46.500000,0.0,76.454545,PRV51629,0
2478,0.0,0.0,3390,678.000000,0.200000,0.200000,0.0,76.800000,PRV54809,0
2302,0.0,0.0,170,56.666667,0.666667,0.666667,0.0,74.666667,PRV52956,0


In [16]:
from sklearn.model_selection import train_test_split

X = data[data.columns[:-2]]

y = data['PotentialFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

print("Accuracy:", accuracy_score(y_test, rfc.predict(X_test)))

Accuracy: 0.9238227146814404


/Users/anmol/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
